In [1]:
import numpy as np

import matplotlib.pyplot as plt

%matplotlib inline
import random

## 2 -Gait Modeal:  LSTMs in Keras: 

 LSTM model that takes as inputs my gait angles features.


In [2]:
import numpy as np
np.random.seed(0)
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation, RNN
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform
import tensorflow as tf
np.random.seed(1)

Using TensorFlow backend.


In [3]:
# load data with Pandas
import pandas as pd
X_all = pd.read_csv("cycles_for_LSTM/covmat_cycles_frames_11t_36f_anglesZX_22persons.csv")
# print things about the files
print(X_all.shape)
print(X_all.head(1))
# Define the T - total frames (in time dimension) and Mat_dim - the dimensionality of a single matrix
T = 11
Mat_dim = 36

(547, 398)
   1  1.1  1.6799  1.2544  1.0591   3.102   2.619  1.9555  0.43623  0.36831  \
0  1    1  1.4158  3.4054  1.5435  3.3255  1.5072  3.6255  0.63371  0.28722   

     ...      0.029403  0.059589  -0.31565  0.010169  0.070043  -0.36121  \
0    ...      0.027058  0.016473  -0.66958 -0.047546  0.005244  -0.72137   

   -0.020401  -0.084859  0.52289  -0.062316  
0  -0.073476   -0.12972   1.1659   0.051024  

[1 rows x 398 columns]


In [4]:
def Reshape_to_input(X, T, Mat_dim):
    """
    reshapes data
    """
   
    [Num_examples, dim] = X.shape
    input_X = X[:,2:].reshape(Num_examples, Mat_dim, T)
    
    return input_X
    

In [5]:
def NormalizeTowardsMean(X):
   # print(X.shape)
    "# Preprocessing: Subtract the mean feature\n",
    mean_feat = np.mean(X, axis=0, keepdims=True)
   # print(mean_feat.shape)
    X -= mean_feat
    # Preprocessing: Divide by standard deviation. This ensures that each feature\n",
    # has roughly the same scale.\n",
    std_feat = np.std(X, axis=0, keepdims=True)
    X /= std_feat
    
    return X



In [6]:
def Random_Selection_train_test (X_all, N):
#    import random
    " This fucntion takes return train and test samples for the LSTM: X dataset, N - number of training samples (i.e persons)"
    Persons = X_all[:,1]
    uniquePersons = np.unique(Persons)
    random.shuffle(uniquePersons)
    num_dim = np.shape(X_all)[1]
    X_train = np.array([], dtype=np.int64).reshape(0,num_dim)
    Y_train = np.array([])
    X_test = np.array([])
    Y_test = np.array([])
    Persons_train = np.array([])
    Persons_test = np.array([])
    #print(X_train.shape)
    for Person in range(0,N):
        indexes = np.where(X_all[:,1]==uniquePersons[Person])
        #print(X_all[[indexes], :][0,0,:,:].shape)
        Person_X_val = X_all[[indexes], :][0,0,:,:]
        X_train= np.append(X_train,Person_X_val)
        label =  np.full((indexes[0].size),uniquePersons[Person])
        Y_train = np.append(Y_train, X_all[[indexes], 0])
        Persons_train = np.append(Persons_train, label, axis = 0)
    X_train=X_train.reshape(Y_train.size, num_dim)    
    for Person in range(N,uniquePersons.size):
        indexes = np.where(X_all[:,1]==uniquePersons[Person])
        X_test = np.append(X_test, X_all[[indexes], :][0,0,:,:])
        label =  np.full((indexes[0].size),uniquePersons[Person])
        Y_test =  np.append(Y_test, X_all[[indexes], 0])
        Persons_test = np.append(Persons_test, label, axis = 0)
    X_test=X_test.reshape(Y_test.size, num_dim) 
    print('Persons for train: ')
    print(np.unique(Persons_train))
    print('Persons for test: ')
    print(np.unique(Persons_test))
    return X_train, Y_train, X_test, Y_test
            
            


In [7]:
def convert_to_one_hot(inputY, C):
    N= inputY.size
    Y=np.zeros((N,C))
    for i in range (0, inputY.size):
        Y[i, int(inputY[i]-1)] = 1
        
    
    return Y


In [8]:
X_train, Y_train, X_test, Y_test = Random_Selection_train_test (X_all.values, 14)

Persons for train: 
[  3.   4.   5.   6.   7.   8.   9.  10.  11.  15.  16.  19.  21.  22.]
Persons for test: 
[  1.   2.  12.  13.  14.  17.  18.  20.]


In [91]:
X_train = NormalizeTowardsMean(X_train)
X_test = NormalizeTowardsMean(X_test)


(336, 398)
(1, 398)
(211, 398)
(1, 398)


### Keras and mini-batching 


In [9]:
def Gait_model(data):
    """
    Function creating the Gait model's graph.
    
    Arguments:
    input - data

    Returns:
    model -- a model instance in Keras
    """
   
    ### START CODE HERE ###
    gait_data = Input(data.shape, dtype='float32')
 
    # Propagate the embeddings through an LSTM layer with 128-dimensional hidden state
    # Be careful, the returned output should be a batch of sequences.

    X = LSTM(128, return_sequences=True)( gait_data)
    # Add dropout with a probability of 0.5
    X = Dropout(0.5)(X)
    # Propagate X trough another LSTM layer with 128-dimensional hidden state
    # Be careful, the returned output should be a single hidden state, not a batch of sequences.
   
    # Propagate X trough another LSTM layer with 128-dimensional hidden state
    X = LSTM(128, return_sequences=False)(X)
    # Add dropout with a probability of 0.5
    X = Dropout(0.5)(X)
    # Propagate X through a Dense layer with softmax activation to get back a batch of 5-dimensional vectors.
    X = Dense(3)(X)
    # Add a softmax activation
    X = Activation('softmax')(X)
    
    # Create Model instance which converts sentence_indices into X.
    model = Model(inputs=gait_data, outputs=X)
    
    ### END CODE HERE ###
    
    return model

In [10]:
# prepare the data
X_train = Reshape_to_input(X_train, T, Mat_dim)
X_test = Reshape_to_input(X_test,T, Mat_dim)

Run the following cell to create your model and check its summary. Because all sentences in the dataset are less than 10 words, we chose `max_len = 10`.  You should see your architecture, it uses "20,223,927" parameters, of which 20,000,050 (the word embeddings) are non-trainable, and the remaining 223,877 are. Because our vocabulary size has 400,001 words (with valid indices from 0 to 400,000) there are 400,001\*50 = 20,000,050 non-trainable parameters. 

In [11]:
print(X_train.shape)
model = Gait_model((X_train[0,:,:]))
model.summary()

(354, 36, 11)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 36, 11)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 36, 128)           71680     
_________________________________________________________________
dropout_1 (Dropout)          (None, 36, 128)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 387       
_________________________________________________________________
activation_1 (Activation)    (None, 3)                 0      

As usual, after creating your model in Keras, you need to compile it and define what loss, optimizer and metrics your are want to use. Compile your model using `categorical_crossentropy` loss, `adam` optimizer and `['accuracy']` metrics:

In [14]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

It's time to train your model. Your Emojifier-V2 `model` takes as input an array of shape (`m`, `max_len`) and outputs probability vectors of shape (`m`, `number of classes`). We thus have to convert X_train (array of sentences as strings) to X_train_indices (array of sentences as list of word indices), and Y_train (labels as indices) to Y_train_oh (labels as one-hot vectors).

In [15]:
#X_train_indices = sentences_to_indices(X_train, word_to_index, maxLen)
Y_train_oh = convert_to_one_hot(Y_train, C = 3)
Y_test_oh = convert_to_one_hot(Y_test, C = 3)

Fit the Keras model on `X_train_indices` and `Y_train_oh`. We will use `epochs = 50` and `batch_size = 32`.

In [16]:
model.fit(X_train, Y_train_oh, epochs = 25 , batch_size = 25, shuffle=True)

Epoch 1/25
354/354 [==============================] - 12s 34ms/step - loss: 1.0095 - acc: 0.5282
Epoch 2/25
354/354 [==============================] - 2s 5ms/step - loss: 0.7653 - acc: 0.6356
Epoch 3/25
354/354 [==============================] - 1s 4ms/step - loss: 0.6741 - acc: 0.7062
Epoch 4/25
354/354 [==============================] - 1s 4ms/step - loss: 0.5817 - acc: 0.7345
Epoch 5/25
354/354 [==============================] - 1s 4ms/step - loss: 0.4830 - acc: 0.7853
Epoch 6/25
354/354 [==============================] - 2s 4ms/step - loss: 0.4016 - acc: 0.8333
Epoch 7/25
354/354 [==============================] - 2s 4ms/step - loss: 0.3229 - acc: 0.8983
Epoch 8/25
354/354 [==============================] - 2s 4ms/step - loss: 0.2605 - acc: 0.8898
Epoch 9/25
354/354 [==============================] - 2s 4ms/step - loss: 0.2056 - acc: 0.9209
Epoch 10/25
354/354 [==============================] - 1s 4ms/step - loss: 0.1649 - acc: 0.9407
Epoch 11/25
354/354 [==========================

Your model should perform close to **100% accuracy** on the training set. The exact accuracy you get may be a little different. Run the following cell to evaluate your model on the test set. 

In [17]:
#X_test_indices = sentences_to_indices(X_test, word_to_index, max_len = maxLen)
#Y_test_oh = convert_to_one_hot(Y_test, C = 5)
loss, acc = model.evaluate(X_test, Y_test_oh)
print()
print("Test accuracy = ", acc)

193/193 [==============================] - 1s 6ms/step

Test accuracy =  0.673575129534


In [18]:
trials = 5
Accuracy = X_train = np.zeros(shape=(trials))

for i in range(0,trials):
    X_train, Y_train, X_test, Y_test = Random_Selection_train_test (X_all.values, 14)
    X_train = NormalizeTowardsMean(X_train)
    X_test = NormalizeTowardsMean(X_test)

    Y_train_oh = convert_to_one_hot(Y_train, C = 3)
    Y_test_oh = convert_to_one_hot(Y_test, C = 3)
    X_train = Reshape_to_input(X_train, T, Mat_dim)
    X_test = Reshape_to_input(X_test, T, Mat_dim)
    model = Gait_model((X_train[0,:,:]))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(X_train, Y_train_oh, epochs = 20, batch_size = 35, shuffle=True)
    loss, acc = model.evaluate(X_test, Y_test_oh)
    Accuracy[i]=acc
    
print(Accuracy)    

Persons for train: 
[  2.   3.   4.   5.   6.   7.  10.  11.  12.  14.  15.  18.  19.  22.]
Persons for test: 
[  1.   8.   9.  13.  16.  17.  20.  21.]
Epoch 1/20
353/353 [==============================] - 12s 35ms/step - loss: 1.0721 - acc: 0.4986
Epoch 2/20
353/353 [==============================] - 2s 4ms/step - loss: 0.9825 - acc: 0.5467
Epoch 3/20
353/353 [==============================] - 1s 3ms/step - loss: 0.8969 - acc: 0.6204
Epoch 4/20
353/353 [==============================] - 1s 3ms/step - loss: 0.8191 - acc: 0.6601
Epoch 5/20
353/353 [==============================] - 1s 3ms/step - loss: 0.7916 - acc: 0.6487
Epoch 6/20
353/353 [==============================] - 1s 3ms/step - loss: 0.7448 - acc: 0.6997
Epoch 7/20
353/353 [==============================] - 1s 3ms/step - loss: 0.7025 - acc: 0.7195
Epoch 8/20
353/353 [==============================] - 1s 3ms/step - loss: 0.6820 - acc: 0.6941
Epoch 9/20
353/353 [==============================] - 1s 3ms/step - loss: 0.6009 - ac

360/360 [==============================] - 1s 3ms/step - loss: 0.3194 - acc: 0.8694
Epoch 19/20
360/360 [==============================] - 1s 3ms/step - loss: 0.2776 - acc: 0.8972
Epoch 20/20
187/187 [==============================] - 1s 3ms/step
Persons for train: 
[  1.   3.   6.   8.   9.  11.  13.  16.  17.  18.  19.  20.  21.  22.]
Persons for test: 
[  2.   4.   5.   7.  10.  12.  14.  15.]
Epoch 1/20
342/342 [==============================] - 12s 37ms/step - loss: 1.0556 - acc: 0.4737
Epoch 2/20
342/342 [==============================] - 2s 5ms/step - loss: 0.8769 - acc: 0.5614
Epoch 3/20
342/342 [==============================] - 1s 3ms/step - loss: 0.7880 - acc: 0.6345
Epoch 4/20
342/342 [==============================] - 1s 3ms/step - loss: 0.7551 - acc: 0.6784
Epoch 5/20
342/342 [==============================] - 1s 3ms/step - loss: 0.7080 - acc: 0.6784
Epoch 6/20
342/342 [==============================] - 1s 3ms/step - loss: 0.6632 - acc: 0.7193
Epoch 7/20
342/342 [========

In [108]:

print(Accuracy)  
#print(Y_train_oh)


[ 0.56544502  0.52849741  0.56632653  0.53170732  0.555     ]


You should get a test accuracy between 80% and 95%. Run the cell below to see the mislabelled examples. 